In [1]:
!pip install geopandas

import folium
import json
import requests
import geopandas as gpd
import pandas as pd

     |████████████████████████████████| 1.0MB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 6.5MB 9.6MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 10.4MB/s eta 0:00:01
     |████████████████████████████████| 15.3MB 22.4MB/s eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6


In [2]:
from shapely import speedups
#speedups.enabled # chk if the result is true
# If false, run this line:
#shapely.speedups.enable()

In [3]:
dxb_communities = gpd.read_file("Community.geojson") #.\Datasets\Community.geojson
dxb_communities.head()

,Name,description,CNAME_E,CNAME_A,OBJECTID,LABEL_E,LABEL_A,C_PREFIX_E,C_PREFIX_A,COMMUNITY_,COMMUNITY1,DGIS_ID,COMM_NUM,NDGIS_ID,SHAPE_AREA,SHAPE_LEN,geometry
0,kml_2,None,AL BADA',البدع,148,AL BADA',البدع,Community:,,AL BADA' - 333,البدع - 333,0000065,333,65,1984700.5385,7946.07017526,"POLYGON ((55.27568 25.23722, 55.27591 25.23708..."
1,kml_3,None,CORNICHE DEIRA,كورنيش ديره,149,CORNICHE DEIRA,كورنيش ديره,Community:,,CORNICHE DEIRA - 121,كورنيش ديره - 121,0000009,121,9,870784.242915,5165.84244281,"POLYGON ((55.32532 25.29458, 55.32631 25.29370..."
2,kml_4,None,AL KHWANEEJ SECOND,الخوانيج الثانية,150,AL KHWANEEJ SECOND,الخوانيج الثانية,Community:,,AL KHWANEEJ SECOND - 282,الخوانيج الثانية - 282,0000056,282,56,12694068.2779,14582.3515084,"POLYGON ((55.54936 25.23110, 55.54686 25.22956..."
3,kml_6,None,AL SAFA FIRST,الصفا الأولى,152,AL SAFA FIRST,الصفا الأولى,Community:,,AL SAFA FIRST - 353,الصفا الأولى - 353,0000090,353,90,2430428.562,6705.66287921,"POLYGON ((55.23987 25.18679, 55.24066 25.18623..."
4,kml_7,None,NAZWAH,نزوه,153,NAZWAH,نزوه,Community:,,NAZWAH - 736,نزوه - 736,0000201,736,201,13058825.2473,15567.1396913,"POLYGON ((55.66182 24.99817, 55.66082 24.99917..."


In [4]:
communities_cols = ['CNAME_E', 'COMM_NUM', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry']
dxb_communities_concise = dxb_communities.loc[:, communities_cols]
dxb_communities_concise.head()

,CNAME_E,COMM_NUM,SHAPE_AREA,SHAPE_LEN,geometry
0,AL BADA',333,1984700.5385,7946.07017526,"POLYGON ((55.27568 25.23722, 55.27591 25.23708..."
1,CORNICHE DEIRA,121,870784.242915,5165.84244281,"POLYGON ((55.32532 25.29458, 55.32631 25.29370..."
2,AL KHWANEEJ SECOND,282,12694068.2779,14582.3515084,"POLYGON ((55.54936 25.23110, 55.54686 25.22956..."
3,AL SAFA FIRST,353,2430428.562,6705.66287921,"POLYGON ((55.23987 25.18679, 55.24066 25.18623..."
4,NAZWAH,736,13058825.2473,15567.1396913,"POLYGON ((55.66182 24.99817, 55.66082 24.99917..."


In [5]:
dxb_healthcare_venues=pd.read_csv("Dxb_Healthcare_Venues_Cleaned.csv") # Removed Veternery, buildings etc
dxb_healthcare_venues.rename(columns = {'Venue Latitude':'Latitude','Venue Longitude':'Longitude'}, inplace = True)
dxb_healthcare_venues.head()

,Venue ID,Venue,Latitude,Longitude,Venue Category
0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital
1,4bf1866b52bda593a9a6b1b7,Emirates Hospital (مستشفى الامارات),25.197293,55.237570,Medical Center
2,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital
3,4b51a09af964a520245127e3,Dubai Healthcare City مدينة دبي الطبية,25.232799,55.321726,Hospital
4,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital


In [6]:
# creating Geodataframe for Healthcare Venue Points
dxb_healthcare_venues_gdf = gpd.GeoDataFrame(dxb_healthcare_venues, geometry=gpd.points_from_xy(dxb_healthcare_venues.Longitude, dxb_healthcare_venues.Latitude))
dxb_healthcare_venues_gdf.crs = "EPSG:4326"
dxb_healthcare_venues_gdf.head()

,Venue ID,Venue,Latitude,Longitude,Venue Category,geometry
0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital,POINT (55.26914 25.23039)
1,4bf1866b52bda593a9a6b1b7,Emirates Hospital (مستشفى الامارات),25.197293,55.237570,Medical Center,POINT (55.23757 25.19729)
2,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital,POINT (55.24280 25.18339)
3,4b51a09af964a520245127e3,Dubai Healthcare City مدينة دبي الطبية,25.232799,55.321726,Hospital,POINT (55.32173 25.23280)
4,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital,POINT (55.26383 25.22196)


### Testing for 1 community

In [7]:
test1_community = dxb_communities_concise.loc[dxb_communities_concise['COMM_NUM']=='241']
test1_community = test1_community.reset_index(drop=True)
test1_community.head()

,CNAME_E,COMM_NUM,SHAPE_AREA,SHAPE_LEN,geometry
0,AL NAHDA SECOND,241,2211570.57,5984.62489457,"POLYGON ((55.38842 25.29573, 55.38783 25.29487..."


In [8]:
pip_mask = dxb_healthcare_venues_gdf.within(test1_community.at[0, 'geometry'])
#print(pip_mask)
pip_data = dxb_healthcare_venues_gdf.loc[pip_mask]
pip_data

,Venue ID,Venue,Latitude,Longitude,Venue Category,geometry
68,4be6b6022468c9282e9a0143,NMC Speciality Hospital,25.290144,55.369405,Hospital,POINT (55.36940 25.29014)
75,4c37226e1e06d13a7f51753e,Zulekha Hospital,25.291293,55.384109,Hospital,POINT (55.38411 25.29129)


### Running Point (Healthcare Venues)  in Polygon (Communities) for all of Dubai

In [9]:
dxb_healthcare_venues_incommunity_gdf = dxb_healthcare_venues_gdf[:0]
#dxb_healthcare_venues_incommunity_gdf.insert(len(dxb_healthcare_venues_incommunity_gdf), "COMM_NUM",[] , True)
dxb_healthcare_venues_incommunity_gdf["COMM_NUM"]=''

# print('dxb_healthcare_venues_incommunity_gdf')
# print(dxb_healthcare_venues_incommunity_gdf, "\n")

#for i in range (len(dxb_communities_concise)) :
for i in range (len(dxb_communities_concise)) :
            #i=1
#            print('this is the ' + str(i) + ' th run',"\n")
            test_community = dxb_communities_concise[i:i+1]
            test_community = test_community.reset_index(drop=True)

#             print('test_community')
#            print(test_community['COMM_NUM'], "\n")

            pip_mask = dxb_healthcare_venues_gdf.within(test_community.at[0, 'geometry'])
            pip_data = dxb_healthcare_venues_gdf.loc[pip_mask]
            pip_data ["COMM_NUM"] = test_community.loc[0,'COMM_NUM']

#            print('pip_data')
#            print(pip_data['Venue'], "\n")

            dxb_healthcare_venues_incommunity_gdf = dxb_healthcare_venues_incommunity_gdf.append(pip_data)

#            print('dxb_healthcare_venues_incommunity_gdf')
#            print(dxb_healthcare_venues_incommunity_gdf['COMM_NUM'],dxb_healthcare_venues_incommunity_gdf['Venue'], "\n")



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [11]:
dxb_healthcare_venues_incommunity_gdf=dxb_healthcare_venues_incommunity_gdf.reset_index(drop=True)
dxb_healthcare_venues_incommunity_gdf

,Venue ID,Venue,Latitude,Longitude,Venue Category,geometry,COMM_NUM
0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital,POINT (55.26914 25.23039),333
1,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital,POINT (55.26383 25.22196),333
2,514705f3e4b0da934ca8ef83,Al Noor Polyclinic,25.229366,55.273964,Hospital,POINT (55.27396 25.22937),333
3,4bbc246bafe1b713fc21304b,Belhoul European Hospital,25.237432,55.275028,Hospital,POINT (55.27503 25.23743),333
4,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital,POINT (55.24280 25.18339),353
...,...,...,...,...,...,...,...
127,5cb2d03ac97f2800396be0e9,King’s College Hospital - Dubai Hills,25.112392,55.254775,Hospital,POINT (55.25478 25.11239),631
128,51a1df79501987a82027628e,burjeel hospital for advance surgeries-tarabic...,25.146749,55.223495,Hospital,POINT (55.22349 25.14675),364
129,4f9bf32be4b0de6267893b8a,Mercato Clinic,25.217103,55.253267,Hospital,POINT (55.25327 25.21710),332
130,5bf4e4b51f8ed6002c643ec3,Mediclinic Deira,25.251147,55.335197,Hospital,POINT (55.33520 25.25115),129


### All Healthcare Venues within Dubai identified and flagged with Community No.
### Now transfer to a data file.

In [12]:
dxb_healthcare_venues_incommunity_gdf.to_csv('Healthcare_Venues_in_Communities.csv')

In [92]:
# m = folium.Map(
#     location=[25.2, 55.2],
#     tiles="openstreetmap",
#     zoom_start=9,
#     )
# # folium.TileLayer('Mapbox Control Room').add_to(m)
# # folium.TileLayer('stamen toner').add_to(m)

# folium.GeoJson(dxb_healthcare_venues_incommunity_gdf, name="geojson").add_to(m)
# folium.GeoJson(dxb_communities, name="geojson1").add_to(m)

# m